In [ ]:
import psi4
import numpy as np
from helper_CQED_RHF import *

In [ ]:

# options for H2O
psi4_options_dict = {
    "basis": "cc-pVDZ",
    "save_jk": True,
    "scf_type": "pk",
    "e_convergence": 1e-12,
    "d_convergence": 1e-12,
}

psi4.set_options(psi4_options_dict)
# molecule string for H2O
molecule_string = """

0 1
    O      0.000000000000   0.000000000000  -0.068516219320
    H      0.000000000000  -0.790689573744   0.543701060715
    H      0.000000000000   0.790689573744   0.543701060715
no_reorient
symmetry c1
"""

# electric field for H2O - polarized along z-axis with mangitude 0.05 atomic units
lambda_vector = np.array([0.0, 0.0, 0.05])

# molecule string for H2O
mol_str_fx = """

0 1
    O      0.001   0.000000000000  -0.068516219320
    H      0.000000000000  -0.790689573744   0.543701060715
    H      0.000000000000   0.790689573744   0.543701060715
no_reorient
no_com
symmetry c1
"""


# molecule string for H2O
mol_str_bx = """

0 1
    O      -0.001   0.000000000000  -0.068516219320
    H      0.000000000000  -0.790689573744   0.543701060715
    H      0.000000000000   0.790689573744   0.543701060715
no_reorient
no_com
symmetry c1
"""

test_string = """
    O      0.000000000000   0.000000000000  -0.068516219320
    H      0.000000000000  -0.790689573744   0.543701060715
    H      0.000000000000   0.790689573744   0.543701060715
symmetry c1
"""

molecule_string = """
O 0.0 0.0 0.0
H 0.0 0.757 0.587
H 0.0 -0.757 0.587
no_reorient
no_com
symmetry c1
"""

test_mol = psi4.geometry(molecule_string)
et, wfn_t = psi4.energy("scf", return_wfn=True)

In [ ]:
# run a psi4 rhf calculation and return the wavefunction object

# sest up molecule object
molecule = psi4.geometry(molecule_string)
nuc_d = molecule.nuclear_dipole()

# get the nuclear repulsion contribution to the gradient
nuclear_repulsion_gradient = np.asarray(molecule.nuclear_repulsion_energy_deriv1())

# Set up Psi4 options
psi4.set_options(psi4_options_dict)

# run energy calculation and return wavefunction
rhf_e , wfn = psi4.energy('scf', return_wfn=True)

# get the rhf gradient
_expected_gradient = np.asarray(psi4.gradient('scf'))

# compute the QED-RHF energy and density matrix
cqed_dict = cqed_rhf(lambda_vector, molecule_string, psi4_options_dict)

# parse dictionary for ordinary RHF and CQED-RHF energy
_rhf_e = cqed_dict["RHF ENERGY"]
_cqed_rhf_e = cqed_dict["CQED-RHF ENERGY"]

# confirm the rhf energy from this method mmatches psi4
assert np.isclose(_rhf_e, rhf_e)

# parse dictionary for density matrix
_cqed_rhf_D = cqed_dict["CQED-RHF DENSITY MATRIX"]
_cqed_rhf_C = cqed_dict["CQED-RHF C"]
_cqed_rhf_eps = cqed_dict["CQED-RHF EPS"]
_cqed_rhf_F = cqed_dict["CQED-RHF FOCK"]





In [ ]:
wfn.Ca().nph[0][:,:] = psi4.core.Matrix.from_array(_cqed_rhf_C)
wfn.Cb().nph[0][:,:] = psi4.core.Matrix.from_array(_cqed_rhf_C)
wfn.Da().nph[0][:,:] = psi4.core.Matrix.from_array(_cqed_rhf_D)
wfn.Db().nph[0][:,:] = psi4.core.Matrix.from_array(_cqed_rhf_D)
wfn.epsilon_a().nph[0][:] = psi4.core.Vector.from_array(_cqed_rhf_eps)
wfn.epsilon_b().nph[0][:] = psi4.core.Vector.from_array(_cqed_rhf_eps)

In [ ]:
# this is the total rhf gradient with the RHF density, etc, stored in wfn
up_scf_gradient = psi4.core.scfgrad(wfn)

In [ ]:
#### call scfgrad for electron-only part of gradient ####
print(np.asarray(up_scf_gradient))
#[[-1.47661150e-16  2.35977968e-15 -1.54615212e-02]
# [ 2.32684749e-16  1.06983093e-02  7.73076058e-03]
# [-8.50235988e-17 -1.06983093e-02  7.73076058e-03]]

In [ ]:
print(gradient)